### Настройка Airflow

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься настройкой среды исполнения, а сразу начать писать код и работать с Airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4

# Инициализация базы данных
!airflow db init

In [ ]:
# Создадим необходимые папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

In [ ]:
# Создадим пользователя Airflow
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

Поместите в dag.py следующий код.

```python
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.operators.dummy_operator import DummyOperator

dag = DAG('dag',schedule_interval=timedelta(days=1), start_date=days_ago(1))
t1 = DummyOperator(task_id='task_1', dag=dag)
t2 = DummyOperator(task_id='task_2',dag=dag)
t3 = DummyOperator(task_id='task_3',dag=dag)
t4 = DummyOperator(task_id='task_4',dag=dag)
t5 = DummyOperator(task_id='task_5',dag=dag)
t6 = DummyOperator(task_id='task_6',dag=dag)
t7 = DummyOperator(task_id='task_7',dag=dag)

[t1, t2]>>t5
t3>>t6
[t5,t6] >>  t7
t4
```

In [ ]:
# Запуск шедулера
!airflow scheduler -D

In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректной работы веб морды
# в среде Google Colab

!pip install pyngrok
!ngrok authtoken <YOUR TOKEN> # найти его можно https://dashboard.ngrok.com/get-started/setup 

# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/cloud-edge/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http -log=stdout 18273 > /dev/null &

После запуска команды выше, перейдите по адресу в ngrok и подождите  пока появится DAG с именем dag

### Практика

**Весь код необходимо вставлять в dag.py**

#### Xcom


Вставьте следующий код в dag.py, после исполните его в вебе и перейдите на вкладку **Admins->Xcom**

**Задание #1**

Укажите то что написано в поле Task Id

```python
import airflow
from airflow import DAG
from airflow.utils.dates import days_ago
from airflow.operators.python_operator import PythonOperator
 
args = {'owner': 'airflow', 'start_date': days_ago(1), 'provide_context': True }
dag = DAG('dag',schedule_interval='@once', default_args=args)
 
value_2 = {'a': 'b'}
 
def push(**kwargs):
    kwargs['ti'].xcom_push(key='key', value=value_2)
 
def pull(**kwargs):
    print(kwargs['ti'].xcom_pull(key='key', task_ids='push'))
 
push1 = PythonOperator(
    task_id='push', dag=dag, python_callable=push)
pull1 = PythonOperator(
    task_id='pull', dag=dag, python_callable=pull)
 
push1 >> pull1
```

#### Запрос в БД к таблице Xcom

Обратимся к Xcom чтобы убедиться что это простая таблица с несколькими полями

**Задание #2**

Вставьте путь до базы данных в качестве ответа

In [ ]:
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql sqlite:////root//airflow/airflow.db
%sql select * from xcom